In [1]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [5]:
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
from tqdm.auto import tqdm
import re

openai_api_key  = get_secret('openai2')['api_key']

mongodb_connect = get_secret('mongodb')['connection_string']

In [6]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True)

'tweet.id_1'

In [7]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [8]:
tweet_data = []
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
for tweet in result:
    text_without_urls = re.sub(url_pattern, '', tweet['tweet']['text'])
    tweet_data.append({'tweet_id':tweet['tweet']['id'],'tweet_text':text_without_urls.replace('\n','')})

In [9]:

print('Number of tweets: ',len(tweet_data))

Number of tweets:  200


In [10]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)
model="gpt-4o"
temperature=0

def openai_help(prompt, model=model, temperature =temperature ):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

In [11]:
for tweet in tqdm(tweet_data):
  
    prompt = f"""
    What is the sentiment of the following tweet, 
    tweet text: {tweet['tweet_text']}
    return  the result with one word as Positive, Neutral,or Negative
 
    """
#     print(prompt)
    try:
        sentiment_result =openai_help(prompt)
    #     print(sentiment_result)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet_id']},
            {"$set":{'tweet.sentiment':sentiment_result}}
        )
    except:
        pass


  0%|          | 0/200 [00:00<?, ?it/s]

In [12]:
for tweet in tqdm(tweet_data):
  
    prompt = f"""
    Translate the following tweet into Chinese
    tweet text: {tweet['tweet_text']}
 
    """
#     print(prompt)
    try:
        translate_result =openai_help(prompt)
#         print(translate_result)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet_id']},
            {"$set":{'tweet.translate':translate_result}}
        )
    except:
        pass

  0%|          | 0/200 [00:00<?, ?it/s]

In [13]:
for tweet in tqdm(tweet_data):
  
    prompt = f"""
    Detect the emotion in the following tweet, and extract whether the tweet expresses anger.
    Provide the result as True, False, or Unknown. 
    Don't provide any reasoning or other output.
    tweet text: {tweet['tweet_text']}
 
    """
#     print(prompt)
    try:
        emotion_result =openai_help(prompt)
        # print(emotion_result)

        tweet_collection.update_one(
                {'tweet.id':tweet['tweet_id']},
                {"$set":{'tweet.anger':emotion_result}}
            )
    except:
        pass

  0%|          | 0/200 [00:00<?, ?it/s]

In [14]:
for tweet in tqdm(tweet_data):
  
    prompt = f"""
    Identify persons or organizations from the following tweet,
    tweet text: {tweet['tweet_text']},
    format the response as a JSON object with Person and Organization as the keys, and extracted items in a list,
    if no entities is not presented, use "Unknown" in the list.
    Do not wrap the JSON codes in JSON markers
   
    """
#     print(prompt)
    try:
        extract_result =openai_help(prompt)
#        print(extract_result)

        tweet_collection.update_one(
                {'tweet.id':tweet['tweet_id']},
                {"$set":{'tweet.extracted_item':json.loads(extract_result)}}
                )
    except:
        pass

  0%|          | 0/200 [00:00<?, ?it/s]

In [15]:
# Define the batch size
batch_size = 500

start_index = 0


while start_index < len(tweet_data):
    batch = tweet_data[start_index:start_index + batch_size]

    tweet_id_list =[]
    tweet_text_summary =''
    
    for tweet in batch:
        tweet_id_list.append(tweet['tweet_id'])
        tweet_text_summary = tweet_text_summary+'.'+tweet['tweet_text']
        
    prompt = f"""
    Summarize the following tweets in at most 50 words, 
    tweet text: {tweet_text_summary,}
 
    """
#     print(prompt)
    try:
        summary_result =openai_help(prompt)

        tweet_summary = db.tweet_summary 
        tweet_summary.insert_one({'id_list':tweet_id_list,
                            'tweet_text_summary':summary_result})
        print(summary_result,'\n')
    except:
        pass
    start_index += batch_size

The tweets discuss various election-related topics, including media portrayal of candidates, potential impacts of a Trump victory on the crypto market, voter registration issues, crime statistics manipulation, and election integrity concerns. They highlight political tensions, misinformation, and the influence of media and dark money on elections, with a focus on the upcoming U.S. presidential election and its potential consequences. 



In [16]:
mongo_client.close()